<a href="https://colab.research.google.com/github/Hase-U/learn_pytorch_advanced/blob/master/mycode/2_objectdetection/SSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出
commit用

```mycode/2_objectdetection/```

## 2-1物体検出とは
**バウンディングボックス(bounding box)：**物体の位置を表す枠

インプット：画像

アウトプット：
- 画像のどこに物体が存在するかを示すバウンディングボックスの位置と大きさ情報
- 各バウンディングボックスに何が写っているか
- その検出に対する確信度



### SSDの流れ
デフォルトボックスの数がN, クラスが背景を入れてO+1の場合
1. 画像のリサイズ
2. デフォルトボックスを準備(N個)
3. 画像をSSDネットワークに入力(出力 N$\times$(O+1+4(オフセット)))
4. 信頼度上位のデフォルトボックスを抽出
5. オフセット情報による修正と被りの除去
6. 閾値を超える信頼度のもののみを最終出力にする

## 2-2 Datasetの実装
データセットがめちゃ大きくてダウンロードに15分毎回かかるから他にいい方法がないか以下を試した
- ```!wget```して解凍
- ファイルをアップロード
- Google drive でマウント

多分```!wget```を毎回するのが一番早いと思われ

In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_11-May-2012.tar

In [0]:
!ls

In [0]:
from google.colab import drive
drive.mount('/content/drive')